In [ ]:
# !pip install pandas chromadb openai gradio
# !pip install python-dotenv

import pandas as pd
import json
import os
import time 
import ast
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import openai
import random
import gradio as gr
from urllib.parse import urlparse, quote
from dotenv import load_dotenv


load_dotenv()

# --- 2/4: Global Variables (전역 변수 및 설정) ---
RESTAURANT_DB_FILE = "restaurant_summaries_output_ALL.csv"
MENU_DB_FILE = "20251017_TOTAL_MENU.csv"
DB_PERSISTENT_PATH = "./restaurant_db"

# 프로필 DB
PROFILE_DB_FILE = "user_profiles_for_hybrid_search.csv"
RESTAURANT_COLLECTION_NAME = "restaurants"
PROFILE_COLLECTION_NAME = "mock_profiles"

# Gradio가 사용할 전역 DB 변수
df_restaurants = None
df_menus = None
collection = None
profile_collection = None
menu_groups = None

# 500명 평가 데이터 저장용 전역 변수
df_all_user_ratings = None 
df_restaurant_ratings_summary = None 

# DB 재구축 설정
CLEAR_DB_AND_REBUILD = False

# 유사 사용자 추천용 전역 변수
sentence_embedder = None

# LLM API 설정
GPT_API_NAME = "gpt-4.1-mini" 
try:
  client = openai.OpenAI() 
  if not client.api_key:
    client.api_key = os.environ.get("OPENAI_API_KEY")
    if not client.api_key:
      raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  exit()

# --- 2. (메인) 챗봇 시스템 프롬프트 ---
SYSTEM_PROMPT = """
당신은 매우 친절하고 지능적인 한국 여행 도우미 챗봇입니다.
당신의 유일한 임무는 사용자와 자연스러운 대화를 나누며, 13가지 필수 정보를 수집하여 JSON 프로필을 완성하는 것입니다.

[수집해야 할 13개 항목 스키마]
0.  name: (사용자의 이름, 예: "Lucas Fernandez", "Soojin Kim")
1.  age: (예: "10대", "20대", "30대"...)
2.  gender: (예: "남", "여", "기타")
3.  nationality: (예: "미국", "일본", "중국")
4.  travel_type: (예: "가족", "혼자", "친구", "연인")
5.  party_size: (예: 1, 2, 4...)
6.  can_wait: (웨이팅 가능 여부, 예: "O", "X")
7.  budget: (예산 수준, 예: "저", "중", "고")
8.  spicy_ok: (매운 음식 가능 여부, 예: "O", "X")
9.  is_vegetarian: (채식 여부, 예: "O", "X")
10. avoid_ingredients: (절대 불가 식재료, 예: "돼지고기", "견과류", "없음")
11. like_ingredients: (좋아하는 식재료, 예: "닭고기", "해산물", "야채")
12. food_category: (선호 음식 분류, 예: "한식", "일식", "디저트", "상관없음")

[대화 규칙]
1.  대화는 당신이 먼저 시작합니다. 환영 인사와 함께 첫 질문(예: 성함)을 하세요.
2.  항상 한 번에 하나씩만 질문하세요.
3.  사용자의 답변을 분석하여 [현재 프로필]을 업데이트합니다.
4.  업데이트된 프로필을 확인하고, 아직 'null'이거나 수집되지 않은 항목 중 하나를 골라 자연스럽게 다음 질문을 합니다.
5.  모든 13개 항목이 수집되면, "설문이 완료되었습니다! 감사합니다."라는 메시지를 보내고 더 이상 질문하지 마세요.
6.  매우 친절하고 공감하는 톤을 유지하세요.

[필수 출력 포맷]
당신은 *반드시* 다음 JSON 형식으로만 응답해야 합니다.
{
  "updated_profile": {
    "name": "Lucas Fernandez", 
    "age": "20대",
    "gender": "남",
    "nationality": null,
    // ... (13개 항목 모두 포함) ...
  },
  "bot_response": "Lucas님이시군요! 반갑습니다. 혹시 연령대가 어떻게 되시나요?"
}
"""

# (13개 항목 템플릿)
PROFILE_TEMPLATE = {
  "name": None, "age": None, "gender": None, "nationality": None, 
  "travel_type": None, "party_size": None, "can_wait": None, 
  "budget": None, "spicy_ok": None, "is_vegetarian": None, 
  "avoid_ingredients": None, "like_ingredients": None, "food_category": None
}

# --- 3/4: 헬퍼 함수 정의 ---

def load_app_data(store_path, menu_path):
  """
  앱 실행에 필요한 모든 CSV 파일을 로드하여
  2개의 전역 DataFrame을 생성합니다.
  """
  global df_restaurants, df_menus, menu_groups
  
  try:
    print(f"'{store_path}'에서 가게 DB 로드 중...")
    df_restaurants = pd.read_csv(store_path)
    df_restaurants['id'] = df_restaurants['id'].astype(str)
    df_restaurants = df_restaurants.set_index('id')
    print(f"가게 DB {len(df_restaurants)}개 로드 완료.")
    
    print(f"'{menu_path}'에서 메뉴 DB 로드 중...")
    df_menus = pd.read_csv(menu_path)
    df_menus['식당ID'] = df_menus['식당ID'].astype(str)
    menu_groups = df_menus.groupby('식당ID')
    print(f"메뉴 DB {len(df_menus)}개 로드 완료 (그룹화 완료).")
    
    return True

  except FileNotFoundError as e:
    print(f"[오류] 필수 파일 로드 실패: {e}")
    return False
  except Exception as e:
    print(f"[오류] 데이터 로드 실패: {e}")
    return False

def load_and_prepare_data(csv_path):
  """
  restaurant_summaries_output...csv 파일을 로드하고
  '메타데이터' 컬럼을 딕셔너리로 변환합니다.
  """
  print(f"'{csv_path}' 파일 로드 중...")
  try:
    df = pd.read_csv(csv_path)
  except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {csv_path}")
    return None

  def safe_convert_to_dict(x):
    if pd.isna(x) or x == '{}' or x == '':
      return {}
    try:
      return ast.literal_eval(x)
    except Exception as e:
      print(f"메타데이터 파싱 오류: {e} \n데이터: {x}")
      return {"error": "parsing_failed"}

  print("메타데이터 컬럼을 딕셔너리로 변환 중...")
  df['메타데이터'] = df['메타데이터'].apply(safe_convert_to_dict)
  df['RAG텍스트'] = df['RAG텍스트'].fillna('')
  
  print(f"데이터 준비 완료: {len(df)}개")
  return df

def build_vector_db(store_csv_path, profile_csv_path, clear_db=False):
  """
  레스토랑 + 프로필용 ChromaDB 2개 컬렉션 생성/로드
  """
  global collection, profile_collection, sentence_embedder
  
  print("\n--- 2단계: VectorDB 구축/로드 시작 ---")
  
  model_name = "distiluse-base-multilingual-cased-v1"
  sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
  )
  sentence_embedder = sentence_transformer_ef._model
  print(f"  > SentenceTransformer 모델 ('{model_name}')을 전역 'sentence_embedder'에 저장했습니다.")
  
  print(f"'{DB_PERSISTENT_PATH}' 경로에서 Persistent DB 클라이언트를 초기화합니다...")
  client = chromadb.PersistentClient(path=DB_PERSISTENT_PATH)

  if clear_db:
    print(f"[경고] CLEAR_DB_AND_REBUILD=True. 컬렉션 2개(restaurants, mock_profiles)를 삭제합니다.")
    try:
      client.delete_collection(name=RESTAURANT_COLLECTION_NAME)
      print(f"  > '{RESTAURANT_COLLECTION_NAME}' 삭제 완료.")
    except Exception as e:
      print(f"  > '{RESTAURANT_COLLECTION_NAME}' 삭제 실패 (무시): {e}")
    try:
      client.delete_collection(name=PROFILE_COLLECTION_NAME)
      print(f"  > '{PROFILE_COLLECTION_NAME}' 삭제 완료.")
    except Exception as e:
      print(f"  > '{PROFILE_COLLECTION_NAME}' 삭제 실패 (무시): {e}")

  # --- 1. 레스토랑 컬렉션 ---
  try:
    print(f"\n[1/2] 기존 '{RESTAURANT_COLLECTION_NAME}' 컬렉션 로드를 시도합니다...")
    collection = client.get_collection(
      name=RESTAURANT_COLLECTION_NAME,
      embedding_function=sentence_transformer_ef
    )
    print(f"  > 'restaurants' 로드 완료: {collection.count()}개")
  except Exception as e:
    print(f"  > 'restaurants' 컬렉션을 찾을 수 없습니다. (이유: {e})")
    print("  > 새 'restaurants' 컬렉션을 생성하고 데이터 적재를 시작합니다.")
    
    df_for_embedding = load_and_prepare_data(store_csv_path)
    if df_for_embedding is None:
      print("[오류] 'restaurants' DB 적재를 위한 원본 CSV 로드에 실패했습니다.")
      return False

    try:
      collection = client.create_collection(
        name=RESTAURANT_COLLECTION_NAME,
        embedding_function=sentence_transformer_ef
      )
    except Exception as e:
      print(f"[오류] 'restaurants' 컬렉션 생성 실패: {e}")
      return False

    documents_list = df_for_embedding['RAG텍스트'].tolist()
    metadatas_list = df_for_embedding['메타데이터'].tolist() 
    ids_list = df_for_embedding['id'].astype(str).tolist()

    print("  > 'restaurants' 메타데이터 변환 중...")
    processed_metadatas = []
    for metadata_dict in metadatas_list:
      processed_meta_item = {}
      for key, value in metadata_dict.items():
        if value is None:
          processed_meta_item[key] = "" 
        elif isinstance(value, list):
          processed_meta_item[key] = ",".join(map(str, value))
        else:
          processed_meta_item[key] = value
      processed_metadatas.append(processed_meta_item)

    print(f"  > 'restaurants' DB에 {len(ids_list)}개 적재 중 (배치)...")
    BATCH_SIZE = 5000
    for i in range(0, len(ids_list), BATCH_SIZE):
      end_i = min(i + BATCH_SIZE, len(ids_list))
      collection.add(
        documents=documents_list[i:end_i],
        metadatas=processed_metadatas[i:end_i],
        ids=ids_list[i:end_i]
      )
    print(f"  > 'restaurants' 신규 구축 완료: {collection.count()}개")

  # --- 2. 프로필 컬렉션 ---
  try:
    print(f"\n[2/2] 기존 '{PROFILE_COLLECTION_NAME}' 컬렉션 로드를 시도합니다...")
    profile_collection = client.get_collection(
      name=PROFILE_COLLECTION_NAME,
      embedding_function=sentence_transformer_ef
    )
    print(f"  > 'mock_profiles' 로드 완료: {profile_collection.count()}개")
  except Exception as e:
    print(f"  > 'mock_profiles' 컬렉션을 찾을 수 없습니다. (이유: {e})")
    print("  > 새 'mock_profiles' 컬렉션을 생성하고 데이터 적재를 시작합니다.")
    
    try:
      df_profiles = pd.read_csv(profile_csv_path)
      df_profiles = df_profiles.dropna(subset=['rag_query_text', 'user_id'])
      print(f"  > '{profile_csv_path}' 파일 로드 완료: {len(df_profiles)}개 프로필")
    except FileNotFoundError:
      print(f"[오류] '{profile_csv_path}' 파일을 찾을 수 없습니다.")
      return False
    except Exception as e:
      print(f"[오류] 프로필 파일 로드 실패: {e}")
      return False

    try:
      profile_collection = client.create_collection(
        name=PROFILE_COLLECTION_NAME,
        embedding_function=sentence_transformer_ef
      )
    except Exception as e:
      print(f"[오류] 'mock_profiles' 컬렉션 생성 실패: {e}")
      return False

    profile_docs = df_profiles['rag_query_text'].tolist()
    profile_ids = df_profiles['user_id'].astype(str).tolist()
    profile_metas = [{'user_id': uid} for uid in profile_ids]

    print(f"  > 'mock_profiles' DB에 {len(profile_ids)}개 적재 중...")
    profile_collection.add(
      documents=profile_docs,
      metadatas=profile_metas,
      ids=profile_ids
    )
    print(f"  > 'mock_profiles' 신규 구축 완료: {profile_collection.count()}개")

  print(f"--- 2단계: VectorDB 2개 컬렉션 로드/구축 완료 ---")
  return True

def call_gpt4o(chat_messages, current_profile):
  """gpt-4.1-mini API 호출 + JSON 파싱"""
  
  system_message_with_profile = f"""
  {SYSTEM_PROMPT}
  [현재까지 수집된 프로필]
  {json.dumps(current_profile, indent=2, ensure_ascii=False)}
  [대화 기록]
  (대화 기록은 아래 메시지 리스트에 포함되어 있습니다)
  """
  
  messages_for_api = [
    {"role": "system", "content": system_message_with_profile}
  ]
  messages_for_api.extend(chat_messages)

  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=messages_for_api,
      response_format={"type": "json_object"}, 
      temperature=0.7
    )
    
    response_content = response.choices[0].message.content
    response_data = json.loads(response_content)
    
    bot_message = response_data.get("bot_response", "오류가 발생했습니다.")
    updated_profile = response_data.get("updated_profile", current_profile)
    
    return bot_message, updated_profile
    
  except Exception as e:
    print(f"API 호출 또는 JSON 파싱 오류: {e}")
    error_message = f"죄송합니다. 챗봇 응답 생성 중 오류가 발생했습니다: {e}"
    return error_message, current_profile

def start_chat():
  """
  채팅방이 처음 로드될 때 실행.
  gpt-4.1-mini를 호출하여 첫 번째 질문을 받는다.
  """
  try:
    initial_profile = PROFILE_TEMPLATE.copy()
    
    bot_message, updated_profile = call_gpt4o(
      chat_messages=[], 
      current_profile=initial_profile
    )
    
    gradio_history = [{"role": "assistant", "content": bot_message}]
    llm_history = [{"role": "assistant", "content": bot_message}]
    
    # user_profile_row_state는 아직 없음
    return gradio_history, llm_history, updated_profile, False, None

  except Exception as e:
    print(f"start_chat에서 API 호출 실패: {e}")
    error_msg = f"챗봇 초기화에 실패했습니다. (API 키 오류일 수 있습니다): {e}"
    return [{"role": "assistant", "content": error_msg}], [], PROFILE_TEMPLATE.copy(), False, None

def generate_profile_summary(profile_data):
  """
  완성된 프로필(JSON)을 받아,
  (1) Gradio 채팅용 메시지, (2) CSV 저장용 원본 요약문 텍스트 반환
  """
  profile_str = json.dumps(profile_data, indent=2, ensure_ascii=False)
  
  summary_system_prompt = """
  당신은 JSON 프로필 데이터를 받아서, 그 사람의 입장에서 자신을 소개하는 '구어체' 텍스트로 변환하는 글쓰기 전문가입니다.
  [규칙]
  1. (필수) JSON의 'name' 필드를 사용하여 "안녕하세요! 저는 [name]입니다."로 문장을 시작하세요.
  2. 딱딱한 리스트가 아닌, 하나의 연결된 문단으로 만드세요.
  3. 모든 정보를 포함하되, 자연스럽게 문장에 녹여내세요.
  4. 'party_size'와 'travel_type'을 묶어서 표현하세요.
  5. 'budget'은 "가성비 있는(저렴한)", "적당한", "고급스러운" 등으로 표현하세요.
  """
  
  user_prompt = f"""
  [사용자 프로필 JSON]
  {profile_str}
  위 프로필을 바탕으로 규칙에 맞게 자기소개 글을 작성해주세요.
  """
  
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": summary_system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      temperature=0.7
    )
    
    raw_summary_text = response.choices[0].message.content
    name = profile_data.get('name', '사용자')
    chat_message_html = f"\n\n---\n\n### 🤖 AI가 파악한 {name}님의 프로필\n\n{raw_summary_text}"
    
    return chat_message_html, raw_summary_text
  
  except Exception as e:
    print(f"요약 생성 오류: {e}")
    error_html = "\n\n(프로필 요약 생성에 실패했습니다.)"
    error_text = "(프로필 요약 생성에 실패했습니다.)"
    return error_html, error_text

def create_filter_metadata(profile_data):
  """
  13개 항목 프로필 → 하이브리드 검색용 6개 필터
  """
  filter_dict = {
    "budget_range": profile_data.get('budget', 'N/A'),
    "spicy_available": profile_data.get('spicy_ok', 'N/A'),
    "vegetarian_options": profile_data.get('is_vegetarian', 'N/A'),
    "main_ingredients_list": profile_data.get('like_ingredients', 'N/A'),
    "suitable_for": profile_data.get('travel_type', 'N/A'),
    "food_category": profile_data.get('food_category', 'N/A')
  }
  return filter_dict

def generate_rag_query(user_profile_summary):
  """
  자기소개 요약문 → RAG용 짧은 핵심 쿼리
  """
  print("  > [RAG] LLM을 호출하여 '분위기/성향' 쿼리를 재작성합니다...")
  
  system_prompt = """
  당신은 사용자의 긴 자기소개 텍스트를, 레스토랑 벡터 DB에서 검색하기 위한
  '짧고 핵심적인 쿼리 문장'으로 재작성하는 전문가입니다.
  1. 개인 신상 정보 제거
  2. 예산/매운 정도/재료 같은 팩트 제거
  3. 분위기/상황/경험/성향만 남기기
  """
  
  user_prompt = f"""
  [사용자 자기소개]
  {user_profile_summary}
  
  [재작성된 쿼리]
  """

  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      temperature=0.2
    )
    rewritten_query = response.choices[0].message.content.strip().replace('"', '')
    return rewritten_query
  except Exception as e:
    print(f"  > [오류] 쿼리 재작성 실패: {e}")
    return user_profile_summary[:150]

def build_filters_from_profile(user_filter_dict):
  """
  사용자 필터 dict → ChromaDB where 필터
  """
  db_pre_filter_list = [] 
  
  DB_FILTER_KEYS = ['budget_range', 'spicy_available', 'vegetarian_options']

  for key, value in user_filter_dict.items():
    if value == 'N/A' or not value: 
      continue
      
    if key == 'food_category':
      db_pre_filter_list.append({"high_level_category": value})
    elif key in DB_FILTER_KEYS:
      db_pre_filter_list.append({key: value})
      
  db_pre_filter = {"$and": db_pre_filter_list} if db_pre_filter_list else {}
  
  return db_pre_filter

def format_restaurant_markdown(store_id_str, rank_prefix="추천", rank_index=1):
  """
  단일 식당 Markdown 문자열 생성 (이미지/링크/메뉴 포함)
  """
  global df_restaurants, menu_groups, df_restaurant_ratings_summary
  
  try:
    store_info = df_restaurants.loc[store_id_str]
    store_name = store_info['가게']
    store_address = store_info['주소']
    store_intro = store_info['소개']
    store_image_url = store_info.get('이미지URL', '') 
    
    detail_url = store_info.get('상세URL', '')
    store_y = store_info.get('Y좌표', '')
    store_x = store_info.get('X좌표', '')
    
    try:
      store_category = store_info.get('high_level_category', 'N/A')
    except KeyError:
      store_category = 'N/A' 

    social_proof_string = "" 
    if df_restaurant_ratings_summary is not None and not df_restaurant_ratings_summary.empty:
      try:
        rating_info = df_restaurant_ratings_summary[
          df_restaurant_ratings_summary['restaurant_id'] == store_id_str
        ]
        if not rating_info.empty:
          recommend_count = rating_info['추천'].iloc[0]
          non_recommend_count = rating_info['미추천'].iloc[0]
          social_proof_string = (
            f"**다른 사용자 평가:** 👍 {recommend_count}명 / 👎 {non_recommend_count}명\n\n"
          )
      except Exception as e:
        print(f"[서식 오류] ID {store_id_str} 평가 카운트 조회: {e}")

    image_md_string = ""
    no_image_filename = "img_restaruant_no_image.png"
    if pd.notna(store_image_url) and store_image_url:
      path = urlparse(store_image_url).path
      filename = os.path.basename(path)
      if filename != no_image_filename:
        image_md_string = f"![{store_name} 이미지]({store_image_url})\n\n"
        
    detail_link_md = ""
    if pd.notna(detail_url) and detail_url:
      detail_link_md = f'<a href="{detail_url}" target="_blank">가게 상세정보</a>'

    map_link_md = ""
    if pd.notna(store_y) and pd.notna(store_x) and store_y and store_x:
      store_name_encoded = quote(store_name)
      kakao_map_url = f"https://map.kakao.com/?q={store_name_encoded}&map_type=TYPE_MAP&rq={store_y},{store_x}"
      map_link_md = f'<a href="{kakao_map_url}" target="_blank">카카오맵 길찾기</a>'

    links_md = ""
    if detail_link_md and map_link_md:
      links_md = f"{detail_link_md} | {map_link_md}\n\n"
    elif detail_link_md:
      links_md = f"{detail_link_md}\n\n"
    elif map_link_md:
      links_md = f"{map_link_md}\n\n"

    menu_str = ""
    try:
      menus_df = menu_groups.get_group(store_id_str)
      rep_menus = menus_df[menus_df['대표여부'] == 'Y'].head(3)
      if rep_menus.empty:
        rep_menus = menus_df.head(3)
      for _, menu_row in rep_menus.iterrows():
        menu_str += f"* {menu_row['메뉴']} ({menu_row['가격원문']})\n"
      if not menu_str:
        menu_str = "* (메뉴 정보 없음)\n"
    except KeyError:
      menu_str = "* (메뉴 정보 없음)\n"

    output_md = (
      f"**[{rank_prefix} {rank_index}] {store_name}**\n\n"
      f"{image_md_string}"
      f"{social_proof_string}"
      f"{links_md}"
      f"**위치:** {store_address}\n\n"
      f"**소개:** {store_intro}\n\n"
      f"**음식종류:** {store_category}\n\n"
      f"**주요메뉴:**\n{menu_str}\n"
      f"\n---\n\n"
    )
    return output_md
    
  except KeyError as ke:
     print(f"[서식 오류] ID {store_id_str} (KeyError): {ke}")
     return f"**[{rank_prefix} {rank_index}] ID: {store_id_str}** (상세 정보 조회 실패)\n\n---\n\n"
  except Exception as inner_e:
     print(f"[서식 오류] ID {store_id_str} (Exception): {inner_e}")
     return f"**[{rank_prefix} {rank_index}] ID: {store_id_str}** (상세 정보 조회 실패)\n\n---\n\n"

def get_similar_user_recommendations(
    live_rag_query_text, 
    primary_reco_ids, 
    max_similar_users=1, 
    max_new_recos=2
  ):
  """
  유사 사용자 추천 헬퍼
  """
  global profile_collection, df_all_user_ratings
  
  if profile_collection is None:
    print("[유사 추천] 'profile_collection'이 로드되지 않았습니다.")
    return ""
    
  if df_all_user_ratings is None:
    print("[유사 추천] 'df_all_user_ratings'가 로드되지 않았습니다.")
    return ""

  try:
    results = profile_collection.query(
      query_texts=[live_rag_query_text],
      n_results=max_similar_users
    )
    
    if not results.get('ids', [[]])[0]:
      print("[유사 추천] 유사한 사용자를 찾지 못했습니다.")
      return ""
      
    similar_user_ids = [meta['user_id'] for meta in results['metadatas'][0]]
    print(f"[유사 추천] 찾은 유사 사용자: {similar_user_ids}")

    similar_user_likes = df_all_user_ratings[
      (df_all_user_ratings['user_id'].isin(similar_user_ids)) &
      (df_all_user_ratings['사용자평가'] == '추천')
    ]
    
    if similar_user_likes.empty:
      print("[유사 추천] 유사 사용자가 '추천'한 식당이 없습니다.")
      return ""

    new_recommendations = []
    for store_id in similar_user_likes['restaurant_id'].astype(str):
      if store_id not in primary_reco_ids and store_id not in new_recommendations:
        new_recommendations.append(store_id)
        
    if not new_recommendations:
      print("[유사 추천] 겹치지 않는 추가 추천 식당이 없습니다.")
      return ""
      
    output_secondary_string = (
      f"\n\n---\n\n"
      f"### 🤖 Charlie님과 비슷한 사용자가 추천한 식당\n\n"
    )
    
    recos_to_show = new_recommendations[:max_new_recos]
    print(f"[유사 추천] 추가할 식당: {recos_to_show}")
    
    for i, store_id in enumerate(recos_to_show):
      output_secondary_string += format_restaurant_markdown(
        store_id, 
        rank_prefix="유사 추천", 
        rank_index=i+1
      )
      
    return output_secondary_string
    
  except Exception as e:
    print(f"[오류] 유사 사용자 추천 생성 중 오류: {e}")
    return ""

def run_hybrid_search_with_scoring(user_profile_row, n_results=5):
  """
  RAG + 점수제 검색 후 상위 n_results개 추천
  """
  print("\n--- 3단계: RAG + 점수제(Scoring) 검색 시작 ---")
  
  try:
    user_original_summary = user_profile_row['rag_query_text']
    user_filter_dict = ast.literal_eval(user_profile_row['filter_metadata_json'])
  except Exception as e:
    print(f"[오류] 사용자 프로필 파싱 실패: {e}")
    return "(오류: 사용자 프로필 파싱 실패)"

  user_rag_query = generate_rag_query(user_original_summary)
  db_pre_filter = build_filters_from_profile(user_filter_dict)
  python_post_filter = {key: val.split(',') for key, val in user_filter_dict.items() 
                        if key in ['main_ingredients_list', 'suitable_for'] and val != 'N/A' and val}
  
  REQUEST_N_RESULTS = 50

  print(f"테스트 사용자: {user_profile_row['name']} ({user_profile_row['user_id']})")
  print(f"RAG 쿼리 (재작성됨): '{user_rag_query}'")
  print(f"DB 1차 필터 (ChromaDB): {db_pre_filter}")
  print(f"Python 2차 필터 (점수 계산용): {python_post_filter}")

  try:
    print(f"\n--- 1단계: RAG + 1차 필터 검색 | Top {REQUEST_N_RESULTS}개 찾기 ---")
    
    if db_pre_filter: 
      results = collection.query(
        query_texts=[user_rag_query],
        n_results=REQUEST_N_RESULTS,
        where=db_pre_filter
      )
    else: 
      results = collection.query(
        query_texts=[user_rag_query],
        n_results=REQUEST_N_RESULTS
      )
    
    print(f"--- 1차 검색 완료: {len(results['ids'][0])}개 후보 반환 ---")
    
    if not results.get('ids', [[]])[0]:
      print("  > [필터 완화] 1차 필터 결과 0건. RAG-Only(필터 없음)로 재시도...")
      results = collection.query(
        query_texts=[user_rag_query],
        n_results=REQUEST_N_RESULTS
      )
      print(f"  > RAG-Only 검색 완료: {len(results['ids'][0])}개 후보 반환")
      if not results.get('ids', [[]])[0]:
        print("RAG-Only 검색 결과도 없습니다.")
        return "\n\n(검색 결과: 1차 필터 및 RAG-Only 검색 모두 0건)"
        
    final_results_with_score = []
    print("\n--- 2단계: 점수(Scoring) 계산 시작 (1차 후보 대상) ---")
    
    for i in range(len(results['ids'][0])):
      store_id = results['ids'][0][i]
      rag_distance = results['distances'][0][i] 
      metadata = results['metadatas'][0][i]
      
      filter_score = 0
      
      if user_filter_dict.get('food_category') == metadata.get('high_level_category'):
        filter_score += 3
      if user_filter_dict.get('budget_range') == metadata.get('budget_range'):
        filter_score += 2
      if user_filter_dict.get('spicy_available') == metadata.get('spicy_available'):
        filter_score += 2
      if user_filter_dict.get('vegetarian_options') == metadata.get('vegetarian_options'):
        filter_score += 2

      if 'suitable_for' in python_post_filter:
        if all(req in metadata.get('suitable_for', '') for req in python_post_filter['suitable_for']): 
          filter_score += 1
      if 'main_ingredients_list' in python_post_filter:
        if any(req in metadata.get('main_ingredients_list', '') for req in python_post_filter['main_ingredients_list']): 
          filter_score += 1

      final_results_with_score.append({
        "id": store_id,
        "rag_distance": rag_distance, 
        "filter_score": filter_score, 
        "metadata": metadata
      })
    
    final_results = sorted(
      final_results_with_score, 
      key=lambda x: (-x['filter_score'], x['rag_distance']), 
    )[:n_results]
    
    print("\n--- 3단계: 최종 RAG + 점수제 랭킹 결과 (기본 추천) ---")
    if not final_results:
      return "\n\n(검색 결과: 1/2차 필터를 모두 만족하는 식당을 찾지 못했습니다.)"

    output_string = "\n\n---\n\n### 🤖 Charlie님을 위한 맞춤 식당 추천!\n\n"
    primary_reco_ids = set()

    for i, item in enumerate(final_results):
      store_id_str = item['id']
      primary_reco_ids.add(store_id_str)
      output_string += format_restaurant_markdown(
        store_id_str, 
        rank_prefix="추천", 
        rank_index=i+1
      )
      
    try:
      print("\n--- 4단계: 유사 사용자 기반 추가 추천 검색 ---")
      secondary_reco_string = get_similar_user_recommendations(
        user_original_summary,
        primary_reco_ids
      )
      if secondary_reco_string:
        output_string += secondary_reco_string
    except Exception as e:
      print(f"[경고] 유사 사용자 추천 생성 중 오류: {e}")

  except Exception as e:
    print(f"\n[오류] 쿼리 실행 중 오류 발생: {e}")
    return f"\n\n(검색 중 오류 발생: {e})"
  
  return output_string

# --- Top-K 변경 시 추천 업데이트 함수 ---

def update_recommendations_with_topk(topk_value, user_profile_row_state):
  """
  Top-K 슬라이더 변경 시: 기존 user_profile_row_state 기반으로 상위 N개 다시 추천
  """
  if user_profile_row_state is None:
    return gr.update(
      value="먼저 프로필 설문을 완료하면 추천 결과가 표시됩니다.",
      visible=False
    )

  k = int(topk_value) if topk_value is not None else 5
  reco_md = run_hybrid_search_with_scoring(
    user_profile_row_state,
    n_results=k
  )
  return gr.update(value=reco_md, visible=True)

# --- 메인 챗봇 함수 (Top-K, user_profile_row_state 연동) ---

def chat_survey(
    message,
    gradio_history,
    llm_history,
    current_profile,
    is_completed,
    topk_value,
    user_profile_row_state
  ):
  """
  사용자가 메시지를 입력할 때마다 실행되는 메인 함수.
  프로필이 완성되면 Top-K 값에 맞춰 추천 실행.
  """
  
  gradio_history.append({"role": "user", "content": message})
  llm_history.append({"role": "user", "content": message})
  
  try:
    bot_message, updated_profile = call_gpt4o(llm_history, current_profile)
  except Exception as e:
    print(f"chat_survey에서 API 호출 실패: {e}")
    error_msg = f"API 호출 중 오류가 발생했습니다: {e}"
    gradio_history.append({"role": "assistant", "content": error_msg})
    return (
      gradio_history,
      llm_history,
      current_profile,
      is_completed,
      gr.update(),
      user_profile_row_state
    )

  llm_history.append({"role": "assistant", "content": bot_message})

  final_bot_message = bot_message
  recommendation_string = gr.update() 
  
  profile_is_complete = all(v is not None for v in updated_profile.values())
  
  if profile_is_complete and not is_completed:
    print("--- 프로필 완성! 요약 및 하이브리드 검색을 시작합니다. ---")
    gr.Info("프로필이 완성되었습니다! AI가 요약 및 식당 추천을 생성 중입니다...")
    
    chat_message_html, raw_summary_text = generate_profile_summary(updated_profile)
    filter_dict = create_filter_metadata(updated_profile)
    filter_metadata_json = json.dumps(filter_dict, ensure_ascii=False)
    
    user_profile_row = {
      "name": updated_profile.get("name", "N/A"),
      "user_id": "live_user",
      "rag_query_text": raw_summary_text,
      "filter_metadata_json": filter_metadata_json
    }

    k = int(topk_value) if topk_value is not None else 5

    if 'collection' in globals() and 'df_restaurants' in globals():
      reco_md = run_hybrid_search_with_scoring(
        user_profile_row,
        n_results=k
      )
      recommendation_string = gr.update(value=reco_md, visible=True)
    else:
      recommendation_string = gr.update(
        value="\n\n[오류] DB가 로드되지 않아 추천을 실행할 수 없습니다.",
        visible=True
      )
    
    user_profile_row_state = user_profile_row

    print("\n" + "="*70)
    print(" [하이브리드 검색용 데이터 생성 완료] ")
    print(f"\n[1] name:\n{user_profile_row['name']}")
    print(f"\n[3] filter_metadata_json:\n{filter_metadata_json}")
    print("="*70 + "\n")

    final_bot_message = f"{bot_message}\n{chat_message_html}\n\n👇 아래에서 추천 결과를 확인하세요! 👇"
    is_completed = True 
    print(json.dumps(updated_profile, indent=2, ensure_ascii=False))

  gradio_history.append({"role": "assistant", "content": final_bot_message})
  
  return (
    gradio_history,
    llm_history,
    updated_profile,
    is_completed,
    recommendation_string,
    user_profile_row_state
  )

# --- 8. DB 및 데이터 전역 로드 ---

MOCK_USER_RATINGS_FILE = "recommendation_results_with_ratings.csv"

try:
  data_loaded_ok = load_app_data(RESTAURANT_DB_FILE, MENU_DB_FILE)
  
  ratings_loaded_ok = False
  if data_loaded_ok:
    try:
      print(f"'{MOCK_USER_RATINGS_FILE}'에서 500명 평가 데이터 로드 중...")
      global df_all_user_ratings, df_restaurant_ratings_summary
      df_all_user_ratings = pd.read_csv(MOCK_USER_RATINGS_FILE)
      
      if 'restaurant_id' in df_all_user_ratings.columns:
        df_all_user_ratings['restaurant_id'] = df_all_user_ratings['restaurant_id'].apply(str)
      else:
        print("[경고] 'restaurant_id' 컬럼이 500명 평가 파일에 없습니다.")
        raise KeyError("'restaurant_id' 컬럼 누락")
      
      if 'user_id' in df_all_user_ratings.columns:
        df_all_user_ratings['user_id'] = df_all_user_ratings['user_id'].apply(str)
      else:
        print("[경고] 'user_id' 컬럼이 500명 평가 파일에 없습니다.")
        raise KeyError("'user_id' 컬럼 누락")

      print("  > 식당(restaurant_id)별 '추천', '미추천' 카운트 집계 중...")
      
      valid_ratings = df_all_user_ratings[
        df_all_user_ratings['사용자평가'].isin(['추천', '미추천'])
      ]
      
      ratings_crosstab = pd.crosstab(
        valid_ratings['restaurant_id'], 
        valid_ratings['사용자평가']
      )
      
      if '추천' not in ratings_crosstab.columns:
        ratings_crosstab['추천'] = 0
      if '미추천' not in ratings_crosstab.columns:
        ratings_crosstab['미추천'] = 0
        
      df_restaurant_ratings_summary = ratings_crosstab[['추천', '미추천']].reset_index()
      
      print(f"  > 500명 평가 데이터 집계 완료: {len(df_restaurant_ratings_summary)}개 식당")
      ratings_loaded_ok = True
      
    except FileNotFoundError:
      print(f"[경고] {MOCK_USER_RATINGS_FILE} 파일을 찾을 수 없습니다. (평가 카운트 기능 비활성화)")
      ratings_loaded_ok = True 
    except Exception as e:
      print(f"[경고] 500명 평가 데이터 로드/집계 중 오류: {e} (평가 카운트 기능 비활성화)")
      ratings_loaded_ok = True

  db_load_ok = build_vector_db(
    RESTAURANT_DB_FILE,
    PROFILE_DB_FILE,
    clear_db=CLEAR_DB_AND_REBUILD
  )
  
  if not data_loaded_ok or not db_load_ok:
     print("[치명적 오류] DB 또는 가게 데이터 로드 실패. Gradio를 시작할 수 없습니다.")
     exit() 
    
except Exception as e:
  print(f"[치명적 오류] DB 로드 중 예외 발생: {e}")
  exit()

# --- 9. Gradio UI 인터페이스 빌드 ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
  gr.Markdown("#길따라 맛따라")
  gr.Markdown("AI가 13가지 프로필 정보를 수집하고, 완료되면 맞춤 식당을 추천합니다.")

  # 언어 설정 영역 (아직 기능만 틀)
  with gr.Group():
    gr.Markdown("### 🌐 언어 설정")
    with gr.Row():
      lang_radio = gr.Radio(
        ["한국어 KR", "English US", "日本語 JP", "中文 CN"],
        label="사용 언어 선택",
        value="한국어 KR",
        interactive=True
      )

  # 상태(State) 변수들
  llm_history_state = gr.State(value=[])
  profile_state = gr.State(value=PROFILE_TEMPLATE.copy())
  is_completed_state = gr.State(value=False)
  user_profile_row_state = gr.State(value=None)

  with gr.Tabs():
    # [탭 1] 음식 탐색
    with gr.TabItem("🍽 음식 탐색"):
      with gr.Column():
        chatbot = gr.Chatbot(
          label="서베이 챗봇",
          height=700,
          show_copy_button=True,
          type="messages",
        )

        msg_textbox = gr.Textbox(
          label="답변 입력",
          placeholder="여기에 답변을 입력하고 Enter를 누르세요...",
        )

        # 챗봇 아래에 맞춤 추천 결과 탭
        with gr.Tabs():
          with gr.TabItem("🌟 맞춤 추천 결과"):
            topk_slider = gr.Slider(
              minimum=1,
              maximum=30,
              value=5,
              step=1,
              label="표시 개수 (Top-K)",
            )
            recommendation_output = gr.Markdown(
              label="추천 결과",
              value="...프로필 설문이 완료되면 여기에 추천 결과가 표시됩니다...",
              visible=False,
            )

    # [탭 2] 설정
    with gr.TabItem("⚙️ 설정"):
      with gr.Column():
        gr.Markdown("### ⚙️ 앱 설정 (예시)")
        gr.Markdown(
          "- 이 탭에는 나중에 벡터 DB 리셋, 디버그 옵션, 모델 선택 등을 넣을 수 있습니다.\n"
          "- 현재는 UI 틀만 만들어 둔 상태입니다."
        )
        rebuild_btn = gr.Button("🔁 벡터 DB 다시 빌드 (예시)")
        debug_checkbox = gr.Checkbox(label="디버그 로그 출력 (예시)", value=False)

  # 초기 로드
  demo.load(
    fn=start_chat,
    inputs=None,
    outputs=[chatbot, llm_history_state, profile_state, is_completed_state, user_profile_row_state],
  )

  # 메시지 전송
  msg_textbox.submit(
    fn=chat_survey,
    inputs=[
      msg_textbox,
      chatbot,
      llm_history_state,
      profile_state,
      is_completed_state,
      topk_slider,
      user_profile_row_state,
    ],
    outputs=[
      chatbot,
      llm_history_state,
      profile_state,
      is_completed_state,
      recommendation_output,
      user_profile_row_state,
    ],
  )

  # Enter 후 입력창 비우기
  msg_textbox.submit(lambda: "", inputs=None, outputs=msg_textbox)

  # Top-K 슬라이더 변경 시 추천 재계산
  topk_slider.change(
    fn=update_recommendations_with_topk,
    inputs=[topk_slider, user_profile_row_state],
    outputs=recommendation_output,
  )




'restaurant_summaries_output_ALL.csv'에서 가게 DB 로드 중...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


가게 DB 7713개 로드 완료.
'20251017_TOTAL_MENU.csv'에서 메뉴 DB 로드 중...
메뉴 DB 54625개 로드 완료 (그룹화 완료).
'recommendation_results_with_ratings.csv'에서 500명 평가 데이터 로드 중...
  > 식당(restaurant_id)별 '추천', '미추천' 카운트 집계 중...
  > 500명 평가 데이터 집계 완료: 428개 식당

--- 2단계: VectorDB 구축/로드 시작 ---
  > SentenceTransformer 모델 ('distiluse-base-multilingual-cased-v1')을 전역 'sentence_embedder'에 저장했습니다.
'./restaurant_db' 경로에서 Persistent DB 클라이언트를 초기화합니다...

[1/2] 기존 'restaurants' 컬렉션 로드를 시도합니다...
  > 'restaurants' 로드 완료: 7713개

[2/2] 기존 'mock_profiles' 컬렉션 로드를 시도합니다...
  > 'mock_profiles' 로드 완료: 500개
--- 2단계: VectorDB 2개 컬렉션 로드/구축 완료 ---


In [23]:
# --- 10. 앱 실행 ---
if __name__ == "__main__":
  if 'collection' not in globals() or 'df_restaurants' not in globals() or 'df_menus' not in globals():
     print("[종료] DB가 정상적으로 로드되지 않았습니다. 스크립트를 종료합니다.")
  else:
     print("Gradio 앱을 시작합니다...")
     demo.launch()



Gradio 앱을 시작합니다...
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


--- 프로필 완성! 요약 및 하이브리드 검색을 시작합니다. ---

--- 3단계: RAG + 점수제(Scoring) 검색 시작 ---
  > [RAG] LLM을 호출하여 '분위기/성향' 쿼리를 재작성합니다...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


테스트 사용자: Chloe (live_user)
RAG 쿼리 (재작성됨): '연인과 함께 편안하고 빠르게 즐길 수 있는 다양한 음식과 해산물 중심의 분위기'
DB 1차 필터 (ChromaDB): {'$and': [{'budget_range': '중'}, {'spicy_available': 'O'}, {'vegetarian_options': 'X'}, {'high_level_category': '상관없음'}]}
Python 2차 필터 (점수 계산용): {'main_ingredients_list': ['해산물'], 'suitable_for': ['연인']}

--- 1단계: RAG + 1차 필터 검색 | Top 50개 찾기 ---
--- 1차 검색 완료: 0개 후보 반환 ---
  > [필터 완화] 1차 필터 결과 0건. RAG-Only(필터 없음)로 재시도...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


  > RAG-Only 검색 완료: 50개 후보 반환

--- 2단계: 점수(Scoring) 계산 시작 (1차 후보 대상) ---

--- 3단계: 최종 RAG + 점수제 랭킹 결과 (기본 추천) ---

--- 4단계: 유사 사용자 기반 추가 추천 검색 ---
[유사 추천] 찾은 유사 사용자: ['user_0288']
[유사 추천] 추가할 식당: ['21046', '31371']

 [하이브리드 검색용 데이터 생성 완료] 

[1] name:
Chloe

[3] filter_metadata_json:
{"budget_range": "중", "spicy_available": "O", "vegetarian_options": "X", "main_ingredients_list": "해산물", "suitable_for": "연인", "food_category": "상관없음"}

{
  "name": "Chloe",
  "age": "20대",
  "gender": "여",
  "nationality": "프랑스",
  "travel_type": "연인",
  "party_size": 2,
  "can_wait": "X",
  "budget": "중",
  "spicy_ok": "O",
  "is_vegetarian": "X",
  "avoid_ingredients": "없음",
  "like_ingredients": "해산물",
  "food_category": "상관없음"
}


In [24]:
demo.close()

Closing server running on port: 7863
